## Классификатор текстов

In [2]:
# Импортируем все необходимые библиотеки для работы.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import os
import tensorflow as tf
print(tf.__version__)

import sys
import keras
import keras.backend as K
import json
np.random.seed(42)

2.6.0


In [4]:
df = pd.read_csv("./drive/My Drive/Genre Classification Dataset/train_data.txt", sep=" ::: ", header=None)
df.columns = ["index", "name", "genre", "desc"]

genres_dict = {genre: idx for idx, genre in enumerate(df["genre"].unique())}

df["genre"] = df["genre"].apply(lambda x: genres_dict[x])
print(genres_dict)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


{'drama': 0, 'thriller': 1, 'adult': 2, 'documentary': 3, 'comedy': 4, 'crime': 5, 'reality-tv': 6, 'horror': 7, 'sport': 8, 'animation': 9, 'action': 10, 'fantasy': 11, 'short': 12, 'sci-fi': 13, 'music': 14, 'adventure': 15, 'talk-show': 16, 'western': 17, 'family': 18, 'mystery': 19, 'history': 20, 'news': 21, 'biography': 22, 'romance': 23, 'game-show': 24, 'musical': 25, 'war': 26}


In [5]:
df.head()

,index,name,genre,desc
0,1,Oscar et la dame rose (2009),0,Listening in to a conversation between his doc...
1,2,Cupid (1997),1,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",2,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),0,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),0,The film's title refers not only to the un-rec...


In [6]:
X = df["desc"].values
Yt = df["genre"].values

del df

len(X)

54214

In [7]:
#Загружаем данные, создаем словарь
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(r'\w+')

PAD = "<PAD>"
START = "<START>"
END = "<END>"
UNK = "<UNK>"

stopwords = stopwords.words('english')
print(stopwords)

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def tokenize(line):
    return [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(line.lower()) if w not in stopwords and w.isalnum()]

def generate_vocabs(texts):
    counter = Counter()
    #1. составляем словарь
    for text in texts:
        counter.update(tokenize(text))
    v = [PAD, START, END, UNK] + [x for x, freq in counter.most_common(25000-4)] #отсекаем слишком редкие слова
    vocab = {token: index for index, token in enumerate(v)}
    vocab_inverse = {idx: w for w, idx in vocab.items()}
    return vocab, vocab_inverse

vocab, vocab_inverse = generate_vocabs(X)

len(vocab), vocab_inverse[0], vocab_inverse[1]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'bef

(25000, '<PAD>', '<START>')

In [8]:
#Преобразуем X сразу в токенизацию, с учетом словаря, отбрасываем слова, не входящие в словарь, заменяем сразу отброшенные слова на <UNK>

def prepare_X_vocab(xt, vocab=vocab):
    result = []
    for x in xt:
        tokenized = tokenize(x)
        result.append([START] + [x for x in tokenized if x in vocab] + [END])
    return result

Xt = prepare_X_vocab(X)
Xt[:3]

[['<START>',
  'listening',
  'conversation',
  'doctor',
  'parent',
  'oscar',
  'learns',
  'nobody',
  'courage',
  'tell',
  'week',
  'live',
  'furious',
  'refuse',
  'speak',
  'anyone',
  'except',
  'rose',
  'lady',
  'pink',
  'meet',
  'hospital',
  'stair',
  'christmas',
  'approach',
  'rose',
  'us',
  'fantastical',
  'experience',
  'professional',
  'wrestler',
  'imagination',
  'wit',
  'charm',
  'allow',
  'oscar',
  'live',
  'life',
  'love',
  'full',
  'company',
  'friend',
  'pop',
  'corn',
  'einstein',
  'bacon',
  'childhood',
  'sweetheart',
  'peggy',
  'blue',
  '<END>'],
 ['<START>',
  'brother',
  'sister',
  'past',
  'incestuous',
  'relationship',
  'current',
  'murderous',
  'relationship',
  'murder',
  'woman',
  'reject',
  'murder',
  'woman',
  'get',
  'close',
  '<END>'],
 ['<START>',
  'bus',
  'empty',
  'student',
  'field',
  'trip',
  'museum',
  'natural',
  'history',
  'little',
  'tour',
  'guide',
  'suspect',
  'student',
 

In [9]:
EMBEDDING_DIM = 64
import keras
import keras.layers as L
from gensim.models import Word2Vec

w2vec = Word2Vec(Xt, size=EMBEDDING_DIM, window=5, min_count=1, workers=4)

def get_embedding(word):
    if word in w2vec:
        return w2vec[word]
    else:
        return np.zeros((1,EMBEDDING_DIM))

#Готовим матрицу коэффициентов
embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))
for word, i in vocab.items():
    embedding_vector = np.zeros((1,EMBEDDING_DIM))
    embedding_vector = get_embedding(word)
    embedding_matrix[i] = embedding_vector 

#Готовый слой Керас
embedding_layer = L.Embedding(len(vocab),
                            EMBEDDING_DIM,
                            weights=[embedding_matrix], 
                            trainable=False, 
                            mask_zero=True)
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


(25000, 64)

In [10]:
pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.50.0-py3-none-any.whl size=19414 sha256=4be3334f73717f52469ddbd0654f4f59636929fa7724ad8077797f5f6b4ce9be
  Stored in directory: /root/.cache/pip/wheels/92/7a/a3/231bef5803298e7ec1815215bc0613239cb1e9c03c57b13c14
Successfully built keras-self-attention


In [11]:

MAX_LEN = 1000
N_LSTM = 512
N_EMBED = 64
N_GENRES = 27
N_VOCAB = len(vocab)


def build_model():
    X = L.Input(batch_input_shape=(None, None))
    #e = L.Embedding(N_VOCAB, N_EMBED, mask_zero=True)(X)
    #e = L.SpatialDropout1D(0.2)(e)
    e = embedding_layer(X)
    l1, hf, cf, hb, cb = L.Bidirectional(L.LSTM(units=N_LSTM, return_sequences=False, return_state=True, dropout=0.1))(e)
    lstm_state = L.Concatenate()([l1, hf, hb]) # конкатенрировал выход и хидден-стейт - так обучается быстрее
    #lstm_state = l1
    d1 = L.Dense(1024, activation='relu')(lstm_state)
    d1 = L.Dropout(0.2)(d1) # изначально 0.15
    d2 = L.Dense(256, activation='relu')(d1)
    d2 = L.Dropout(0.2)(d2) # изначально 0.15
    Y = L.Dense(N_GENRES, activation='softmax')(d2)
    return keras.models.Model(inputs=X, outputs=Y)
    
model = build_model()
opt = tf.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     1600000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 1024), (None 2363392     embedding[0][0]                  
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2048)         0           bidirectional[0][0]              
                                                                 bidirectional[0][1]          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [12]:
#Модель Self-Attention

N_LSTM = 256
N_EMBED = 64
N_GENRES = 27
N_VOCAB = len(vocab)

from keras_self_attention import SeqSelfAttention

def build_amodel():
    X = L.Input(batch_input_shape=(None, None))
    e = embedding_layer(X)
    l1, hf, cf, hb, cb = L.Bidirectional(L.LSTM(units=N_LSTM, return_sequences=True, return_state=True))(e)
    
    att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation=None)(l1)
    avg_att = L.GlobalAveragePooling1D()(att)

    #total_state = L.Concatenate()([l1[...,-1, :], hb, hf, avg_att])
    total_state = L.Concatenate()([l1[..., -1, :], avg_att])
    d1 = L.Dense(1024, activation='relu')(total_state)
    d1 = L.Dropout(0.1)(d1) # изначально 0.15
    d2 = L.Dense(256, activation='relu')(d1)
    d2 = L.Dropout(0.1)(d2) # изначально 0.15
    Y = L.Dense(N_GENRES, activation='softmax')(d2)
    return keras.models.Model(inputs=X, outputs=Y)

model = build_amodel()
opt = tf.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     1600000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, None, 512),  657408      embedding[1][0]                  
__________________________________________________________________________________________________
seq_self_attention (SeqSelfAtte (None, None, 512)    262145      bidirectional_1[0][0]            
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
Xt[:3], Yt[:3], len(Xt)

([['<START>',
   'listening',
   'conversation',
   'doctor',
   'parent',
   'oscar',
   'learns',
   'nobody',
   'courage',
   'tell',
   'week',
   'live',
   'furious',
   'refuse',
   'speak',
   'anyone',
   'except',
   'rose',
   'lady',
   'pink',
   'meet',
   'hospital',
   'stair',
   'christmas',
   'approach',
   'rose',
   'us',
   'fantastical',
   'experience',
   'professional',
   'wrestler',
   'imagination',
   'wit',
   'charm',
   'allow',
   'oscar',
   'live',
   'life',
   'love',
   'full',
   'company',
   'friend',
   'pop',
   'corn',
   'einstein',
   'bacon',
   'childhood',
   'sweetheart',
   'peggy',
   'blue',
   '<END>'],
  ['<START>',
   'brother',
   'sister',
   'past',
   'incestuous',
   'relationship',
   'current',
   'murderous',
   'relationship',
   'murder',
   'woman',
   'reject',
   'murder',
   'woman',
   'get',
   'close',
   '<END>'],
  ['<START>',
   'bus',
   'empty',
   'student',
   'field',
   'trip',
   'museum',
   'natural

In [14]:
BATCH_SIZE = 32
MAX_LEN = 1000

def to_text(seq):
    return " ".join([vocab_inverse[idx] for idx in seq])

def to_sequence(text, vocab=vocab):
    #tokens = tokenize(text)
    tokens = text
    return [vocab[w] if w in vocab else vocab[UNK] for w in tokens]

def to_matrix(texts, maxlen=0):
    seqs = [to_sequence(text) for text in texts]
    if maxlen == 0:
        maxlen = min(MAX_LEN, max(list(map(len, seqs))))
    return keras.preprocessing.sequence.pad_sequences(seqs, maxlen=maxlen, dtype='int32', padding='post', truncating='post', value=0)

import random

def shuffleXY(x, y):
    Z = list(zip(x, y))
    random.shuffle(Z)
    return zip(*Z)

def train_gen(x, y):
    offset = 0
    count = BATCH_SIZE
    while True:
        #if offset == 0:
        #    x, y = shuffleXY(x, y)
        xt = to_matrix(x[offset:offset+count])
        yt = y[offset:offset+count]
        yield xt, yt
        offset += count
        if offset >= len(x)//BATCH_SIZE*BATCH_SIZE:
            offset = 0
            
def val_gen(x, y):
    offset = 0
    count = BATCH_SIZE
    while True:
        xv = to_matrix(x[offset:offset+count])
        yv = y[offset:offset+count]
        yield xv, yv
        offset += count
        if offset >= len(x)//BATCH_SIZE*BATCH_SIZE:
            offset = 0

def _schedule(epoch, lr):
    if epoch < 3:
        return 0.001
    else:
        return lr * tf.math.exp(-0.1)

from sklearn.model_selection import train_test_split

Xtt, Xtv, Ytt, Ytv = train_test_split(Xt, Yt, test_size=0.25, random_state=42)

def train_model(initial_epoch, n_epochs, train_gen, val_gen):
    model.fit(train_gen, epochs=n_epochs, validation_data = val_gen, validation_steps=len(Xtv)//BATCH_SIZE,
                steps_per_epoch=len(Xtt)//BATCH_SIZE,  initial_epoch=initial_epoch, 
                callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath='./model.{epoch:03d}.hdf5')
])
    
len(Xtt), len(Xtv)

(40660, 13554)

In [15]:
opt = tf.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [16]:
train_model(0, 10, train_gen(Xtt, Ytt), val_gen(Xtv, Ytv))

Epoch 1/10
1270/1270 [==============================] - 172s 126ms/step - loss: 1.6181 - accuracy: 0.5201 - val_loss: 1.4918 - val_accuracy: 0.5474
Epoch 2/10
1270/1270 [==============================] - 156s 123ms/step - loss: 1.4110 - accuracy: 0.5715 - val_loss: 1.4259 - val_accuracy: 0.5633
Epoch 3/10
1270/1270 [==============================] - 155s 122ms/step - loss: 1.2952 - accuracy: 0.5998 - val_loss: 1.4137 - val_accuracy: 0.5717
Epoch 4/10
1270/1270 [==============================] - 155s 122ms/step - loss: 1.1577 - accuracy: 0.6374 - val_loss: 1.4714 - val_accuracy: 0.5593
Epoch 5/10
1270/1270 [==============================] - 156s 122ms/step - loss: 0.9880 - accuracy: 0.6864 - val_loss: 1.5616 - val_accuracy: 0.5510
Epoch 6/10
1270/1270 [==============================] - 155s 122ms/step - loss: 0.8114 - accuracy: 0.7377 - val_loss: 1.8093 - val_accuracy: 0.5510
Epoch 7/10
1270/1270 [==============================] - 155s 122ms/step - loss: 0.6453 - accuracy: 0.7853 - val_

In [17]:
df = pd.read_csv("./drive/My Drive/Genre Classification Dataset/test_data_solution.txt", sep=" ::: ", header=None)
df.columns = ["index", "name", "genre", "desc"]
df["genre"] = df["genre"].apply(lambda x: genres_dict[x])

Xv = prepare_X_vocab(df["desc"].values)
Yv = df["genre"].values
del df
print(len(Xv))
print(Xv[:3])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


54200
[['<START>', 'love', 'life', 'car', 'apartment', 'job', 'especially', 'girlfriend', 'vespa', 'one', 'day', 'showering', 'vespa', 'run', 'shampoo', 'run', 'across', 'street', 'convenience', 'store', 'buy', 'quick', 'trip', 'minute', 'return', 'vespa', 'gone', 'every', 'trace', 'existence', 'wiped', 'life', 'becomes', 'tortured', 'existence', 'one', 'strange', 'event', 'another', 'occurs', 'confirm', 'mind', 'conspiracy', 'working', 'finding', 'vespa', '<END>'], ['<START>', 'spain', 'march', '1964', 'naughty', 'child', 'three', 'belonging', 'wealthy', 'family', 'since', 'cristina', 'birth', 'feel', 'lost', 'privileged', 'position', 'prince', 'house', 'eight', 'month', 'old', 'sister', 'brother', 'juan', 'eight', 'year', 'old', 'quite', 'spend', 'time', 'committing', 'prank', 'prank', 'causing', 'resulting', 'anger', 'mother', 'nanny', 'old', 'housemaid', 'rest', 'family', 'member', 'two', 'much', 'older', 'brother', 'resigned', 'mother', 'father', 'authoritarian', 'idea', 'many', '

In [18]:
model.evaluate(val_gen(Xv, Yv), steps=len(Xv)//BATCH_SIZE)

1693/1693 [==============================] - 71s 42ms/step - loss: 2.8538 - accuracy: 0.5330


[2.853775978088379, 0.5330219864845276]

**Вывод:**
Из текущего задания, делаю следующий ввыод:
1. Без токенов "Start" и "End" работает значительно хуже.
2. Spatial1DDropout на эмбенддинге не помогает.
3. Замена эмбенддинг слоя на Word2Vec матрицу помогло, но не сильно. Данная модель помогла только в скорости обучения.
4. Конкатенация H-стейта и выхода LSTM - сильно помогает при обучении. Сильное ограничение словаря - не помогает.
5. Различные варианты выбора тренировочных и валидационных данных дают спорные результаты. А так же, различные варианты конструкции выходных слоев, так же дают спорный результат.
Максимальное значение, которое я получил на тестовой выборке: 0.53-0.54.